In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as py
import seaborn as sns

In [4]:
data=pd.read_csv("./covid_19_data.csv")

In [5]:
data.tail()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
3079,3080,03/02/2020,"Umatilla, OR",US,2020-03-02T20:23:16,1.0,0.0,0.0
3080,3081,03/02/2020,From Diamond Princess,Australia,2020-02-29T02:03:10,0.0,0.0,0.0
3081,3082,03/02/2020,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0
3082,3083,03/02/2020,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0
3083,3084,03/02/2020,"Travis, CA (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0


In [6]:
data["LastUpdate"]=pd.to_datetime(data['Last Update'])

In [7]:
# Data Preprocessing
#Removing the last update column

In [8]:
full_list=data.copy()

In [9]:
ncov_data=full_list.drop(["SNo",'LastUpdate'],axis=1)

In [10]:
ncov_data[['Province/State', 'Country/Region']] = ncov_data[['Province/State', 'Country/Region']].fillna('NA')
ncov_data[['Confirmed', 'Deaths', 'Recovered']] = ncov_data[['Confirmed', 'Deaths', 'Recovered']].fillna(0.0)


In [11]:
ncov_data[['Country/Region']]=ncov_data[['Country/Region']].replace('Mainland China','China')

# Exploratory Data Analysis

In [12]:
#Finding the number of countries effected by  the virus.

In [159]:
temp = ncov_data.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max()
temp.style.background_gradient(cmap='Blues')

C:\Users\Alinahuda\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [14]:
#create a new dataframe with max date to get the latest cummulative data
ncovd_latest=ncov_data[ncov_data['ObservationDate']==ncov_data['ObservationDate'].max()]


In [30]:
#Total number of latest cases 
temp = ncovd_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered'].sum()
temp=temp.sort_values(by='Confirmed',ascending=False)
temp.style.background_gradient(cmap='coolwarm')

C:\Users\Alinahuda\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Confirmed,Deaths,Recovered
Country/Region,,,
China,80026.000000,2912.000000,44810.000000
South Korea,4335.000000,28.000000,30.000000
Italy,2036.000000,52.000000,149.000000
Iran,1501.000000,66.000000,291.000000
Others,705.000000,6.000000,10.000000
Japan,274.000000,6.000000,32.000000
France,191.000000,3.000000,12.000000
Germany,159.000000,0.000000,16.000000
Spain,120.000000,0.000000,2.000000


In [16]:
#Number of countries Effected by coronavirus
count=ncovd_latest['Country/Region'].unique()
print("Number of countries effected",len(count))

Number of countries effected 75


In [127]:
#Latest confirmed, recovered death cases
latest=ncovd_latest.groupby('ObservationDate').sum()
latest.style.background_gradient(cmap="viridis")


,Confirmed,Deaths,Recovered
ObservationDate,,,
03/02/2020,90309.000000,3085.000000,45602.000000


In [34]:
#Cases in china
china=ncovd_latest[ncovd_latest['Country/Region']=='China']


In [35]:
china_cases=china.groupby('Province/State').sum()
china_cases=china_cases.sort_values(by='Confirmed',ascending=False)
china_cases.style.background_gradient(cmap='Reds')

,Confirmed,Deaths,Recovered
Province/State,,,
Hubei,67103.000000,2803.000000,33934.000000
Guangdong,1350.000000,7.000000,1059.000000
Henan,1272.000000,22.000000,1205.000000
Zhejiang,1206.000000,1.000000,1069.000000
Hunan,1018.000000,4.000000,887.000000
Anhui,990.000000,6.000000,917.000000
Jiangxi,935.000000,1.000000,850.000000
Shandong,758.000000,6.000000,460.000000
Jiangsu,631.000000,0.000000,543.000000


In [36]:
#Cases in USA
USA=ncovd_latest[ncovd_latest['Country/Region']=='US']


In [37]:
USA_cases=USA.groupby('Province/State').sum()
USA_cases=USA_cases.sort_values(by='Confirmed',ascending=False)
USA_cases.style.background_gradient(cmap='Blues')

,Confirmed,Deaths,Recovered
Province/State,,,
Unassigned Location (From Diamond Princess),45.000000,0.000000,0.000000
"King County, WA",14.000000,5.000000,1.000000
"Santa Clara, CA",9.000000,0.000000,1.000000
"Snohomish County, WA",4.000000,1.000000,0.000000
"Cook County, IL",4.000000,0.000000,2.000000
"Portland, OR",2.000000,0.000000,0.000000
"San Diego County, CA",2.000000,0.000000,1.000000
"San Benito, CA",2.000000,0.000000,0.000000
"Sacramento County, CA",2.000000,0.000000,0.000000


In [81]:
plot=ncovd_latest.groupby(['Country/Region']).sum().reset_index().sort_values(by='Confirmed',ascending=False)

In [79]:
#Plotting confirmed cases 
import plotly.express as px
fig = px.bar(plot,x=plot['Country/Region'],y=plot['Confirmed'], color=plot['Country/Region'],height=400,log_y=True)
fig.show()


In [80]:
#Plottting death cases
fig = px.bar(plot,x=plot['Country/Region'],y=plot['Deaths'], color=plot['Country/Region'],height=400,log_y=True)
fig.show()


# Impact of coronavirus world wide

In [133]:
#Worldwide trend of cases
import matplotlib.pyplot as plt

china_trend=ncov_data.groupby('ObservationDate').sum().reset_index()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig=go.Figure()

# Add traces
fig.add_trace(
    go.Bar(x=china_trend['ObservationDate'],y=china_trend['Confirmed'] ,name="Confirmed",
   marker_color="orange")
)

fig.add_trace(
    go.Bar(x=china_trend['ObservationDate'],y=china_trend['Recovered'], name="Recovered",
   marker_color="Green")
)
fig.add_trace(
    go.Bar(x=china_trend['ObservationDate'],y=china_trend['Deaths'] ,name="Deaths",
   marker_color="Red")
)

# Add figure title
fig.update_layout(
    title_text="Confirmed,Deaths, Recovered Coronavirus cases World wide",
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
)

)



fig.show()



# Confirm cases in China vs other countries


In [135]:
other_countries=ncov_data[ncov_data['Country/Region']!='China'].groupby('ObservationDate').sum().reset_index()
china=ncov_data[ncov_data['Country/Region']=='China'].groupby('ObservationDate').sum().reset_index()

In [167]:
fig=go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=other_countries['ObservationDate'],y=other_countries['Confirmed'] ,name="other_countries",
   marker_color="orange",mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=china['ObservationDate'],y=china['Confirmed'], name="China",
   marker_color="Green",mode='lines+markers')
)


# Add figure title
fig.update_layout(
    title_text="Confirm cases china vs other countries",
    yaxis_type="log",
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
)

)

fig.show()

# Deaths in china vs other countries

In [166]:
fig=go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=other_countries['ObservationDate'],y=other_countries['Deaths'] ,name="other_countries",marker_color="blue",
   mode='lines+markers')
)

fig.add_trace(
    go.Scatter(x=china['ObservationDate'],y=china['Deaths'], name="China",mode='lines+markers',
   marker_color="red")
)


# Add figure title
fig.update_layout(
    title_text="Deaths in china vs other countries",
    yaxis_type="log",
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
)

)

fig.show()

# Recovery in china vs other countries

In [165]:
fig=go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=other_countries['ObservationDate'],y=other_countries['Recovered'] ,name="other_countries",mode='lines+markers'
  )
)

fig.add_trace(
    go.Scatter(x=china['ObservationDate'],y=china['Recovered'], name="China",marker_color="red",mode='lines+markers'
   )
)


# Add figure title
fig.update_layout(
    title_text="Recovery in china vs other countries",
    yaxis_type="log",
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
)

)

fig.show()